# About the project:
+ All of the required answers are printed in the output cells; 
+ ``Step X`` is the corresponding subquestions under each problem;
+ Timer is printed for time-consuming cells and algorithms.

In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import time

In [2]:
boston = load_boston()
X = boston.data
y = boston.target

# Problem 1

## Preprocessing

In [3]:
print(X)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(len(X)-400)/len(X), random_state=42)

In [5]:
print("X_train size: ", len(X_train))
print("X_test size: ", len(X_test))
print("y_train size: ", len(y_train))
print("X_test size: ", len(y_test))

X_train size:  400
X_test size:  106
y_train size:  400
X_test size:  106


In [6]:
scaler = preprocessing.StandardScaler().fit(X_train)
print("mean: ", scaler.mean_)
print("std:", scaler.scale_)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

mean:  [3.5883762e+00 1.1597500e+01 1.0968900e+01 7.2500000e-02 5.5653925e-01
 6.3208700e+00 6.8586500e+01 3.8081225e+00 9.3175000e+00 4.0321500e+02
 1.8305750e+01 3.5706645e+02 1.2420675e+01]
std: [8.88523993e+00 2.31879385e+01 6.89623233e+00 2.59313999e-01
 1.17922423e-01 7.09658614e-01 2.79528025e+01 2.13065150e+00
 8.54498062e+00 1.65614292e+02 2.22788957e+00 9.01651569e+01
 7.08244586e+00]


## Step 1: MSE
$\hat\theta = (A^TA)^{-1}A^Ty$

In [7]:
def MSE(theta_hat, A, y):
    return 1/len(y)*np.sum(np.power(y-np.dot(A, theta_hat),2))

In [24]:
A_train = np.insert(X_train, 0, values = np.array([[1]*len(X_train)]), axis = 1)
A_test = np.insert(X_test, 0, values = np.array([[1]*len(X_test)]), axis = 1)
theta_hat = np.dot(np.dot(np.linalg.inv(np.dot(A_train.T, A_train)),A_train.T),y_train)
y_pred = np.dot(A_test, theta_hat)
print("Mean-Squared Error: ", MSE(theta_hat, A_test, y_test))

Mean-Squared Error:  23.561522590845065


## Step 2: Ridge Regression
$\hat\theta = (A^TA+\lambda I)^{-1}A^Ty$

In [9]:
def ridge_regression(A, y, lambd=0.2):
    I = np.eye(A.shape[1])
    I[0][0] = 0
    return np.dot(np.dot(np.linalg.inv(np.dot(A.T, A)+lambd*I),A.T),y)

### a. Take example of $\lambda = 0.2$

In [10]:
theta_r = ridge_regression(A_train, y_train, lambd=0.2)
print("Mean-Squared Error: ", MSE(theta_r, A_test, y_test))

Mean-Squared Error:  23.60489050802226


### b. Find the best $\lambda$ value
A commom way to find out the best $\lambda$ value is K-folder cross validation. The idea is shown as follows:
+ Randomly divide the data into K equal-sized parts;
+ Leave leave out part $K$ and put it to the side;
+ Fit the model to the other $K-1$ parts (combined) as the training set;
+ Use part $K$ as the holdout set to estimate the prediction error;
+ Repeat this process $K$ times, once each for the different splits of the data;
+ Select the $\lambda$ in which the cross-validation error is smallest.

In [11]:
def Ridge_find_best(X, y, kfold, lambdas, to_print = True):
    kf = KFold(n_splits = kfold, shuffle = True)
    best_lambda = 0
    best_MSE = 100
    for idx, lambda_value in enumerate(lambdas):
        sum = 0
        for train_index, holdout_index in kf.split(X):
            Xtrain, Xholdout = X[train_index], X[holdout_index]
            ytrain, yholdout = y[train_index], y[holdout_index]
            Atrain = np.insert(Xtrain, 0, values = np.array([[1]*len(Xtrain)]), axis = 1)
            Aholdout = np.insert(Xholdout, 0, values = np.array([[1]*len(Xholdout)]), axis = 1)
            theta_kf = ridge_regression(Atrain, ytrain, lambd = lambda_value)
            sum += MSE(theta_kf, Aholdout, yholdout)
        MSE_now = sum/kfold
        if MSE_now < best_MSE:
            print("Current MSE: ", MSE_now, "; Current lambda: ",lambda_value)
            best_MSE = MSE_now
            best_lambda = lambda_value
    if to_print:
        print("===========================")
        print("The best value of lambda is: ",best_lambda)
    return best_lambda

In [26]:
start_time = time.time()
lambda_value = Ridge_find_best(X_train, y_train, 10, np.arange(0,2,0.0001), to_print = True)
print("===========================")
print("This cell takes %s seconds to run." % (time.time() - start_time))

Current MSE:  24.320497059965753 ; Current lambda:  0.0
Current MSE:  23.753618728614672 ; Current lambda:  0.0001
Current MSE:  23.750720799632013 ; Current lambda:  0.0046
Current MSE:  23.502454089840576 ; Current lambda:  0.0059
Current MSE:  23.464372846781185 ; Current lambda:  0.0223
Current MSE:  23.429693170121833 ; Current lambda:  0.045000000000000005
Current MSE:  23.410600378348285 ; Current lambda:  0.0466
Current MSE:  23.10050759573165 ; Current lambda:  0.0664
The best value of lambda is:  0.0664
This cell takes 73.7678771018982 seconds to run.


In [27]:
theta_r = ridge_regression(A_train, y_train, lambd=lambda_value)
print("Mean-Squared Error: ", MSE(theta_r, A_test, y_test))

Mean-Squared Error:  23.573650639588113


### c. Explaination
We find that when $\lambda = 0$, the Mean-Squared Error is the minimum. This is because $\theta_{Linear Regression}$ is the minimizer of MSE by definition. The problem ($A^TA$ is invertible here) has only one minimum and any value other than $\theta_{Linear Regression}$ will have higher MSE on the training dataset.

## Step 3: LASSO Regression

### a. Take example of $\lambda = 0.2$

In [16]:
reg = linear_model.Lasso(alpha = 0.2) # Fill in alpha
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)
print("Coefficients: ", reg.coef_)
print("Mean-Squared Error: %.2f" % mean_squared_error(y_test, y_pred))

Coefficients:  [-0.10131917  0.03459379 -0.00436712  0.         -0.          4.01779234
 -0.01077091 -1.09265737  0.23987994 -0.01316046 -0.74193498  0.01324755
 -0.59334959]
Mean-Squared Error: 24.48


### b. Find the best $\lambda$ value
Another efficient method is to use LassoCV() function directly. It will be faster, but its performace is in terms of R_2 value rather than MSE.

In [17]:
def Lasso_find_best(X, y, kfold, lambdas, to_print = True):
    kf = KFold(n_splits = kfold, shuffle = True)
    best_lambda = 0
    best_MSE = 100
    for idx, lambda_value in enumerate(lambdas):
        sum = 0
        for train_index, holdout_index in kf.split(X):
            Xtrain, Xholdout = X[train_index], X[holdout_index]
            ytrain, yholdout = y[train_index], y[holdout_index]
            reg = linear_model.Lasso(alpha = lambda_value) # Fill in alpha
            reg.fit(Xtrain,ytrain)
            ypred = reg.predict(Xholdout)
            sum += mean_squared_error(yholdout, ypred)
        MSE_now = sum/kfold
        if MSE_now < best_MSE:
            print("Current MSE: ", MSE_now, "; Current lambda: ",lambda_value)
            best_MSE = MSE_now
            best_lambda = lambda_value
    if to_print:
        print("===========================")
        print("The best value of lambda is: ",best_lambda)
    return best_lambda

In [22]:
start_time = time.time()
lambda_value = Lasso_find_best(X_train, y_train, 10, np.arange(0,10,0.001), to_print = True)
print("===========================")
print("This cell takes %s seconds to run." % (time.time() - start_time))

/var/folders/jr/tf9k36ws1956sfj0w8ybnj2m0000gn/T/ipykernel_16152/235148019.py:11: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  reg.fit(Xtrain,ytrain)
/Users/jiayingli/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/jiayingli/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4117.201376801888, tolerance: 3.1884527888888896
  model = cd_fast.enet_coordinate_descent(
/var/folders/jr/tf9k36ws1956sfj0w8ybnj2m0000gn/T/ipykernel_16152/235148019.py:11: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


Current MSE:  24.15305212949115 ; Current lambda:  0.0
Current MSE:  23.93102530886462 ; Current lambda:  0.001
Current MSE:  23.74974571379608 ; Current lambda:  0.009000000000000001
Current MSE:  23.684635138084552 ; Current lambda:  0.012
The best value of lambda is:  0.012
This cell takes 34.62859082221985 seconds to run.


In [23]:
reg = linear_model.Lasso(alpha = lambda_value) # Fill in alpha
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)
print("Coefficients: ", reg.coef_)
print("number of nonzeros in θ: ", np.count_nonzero(reg.coef_))
print("Mean-Squared Error: %.5f" % mean_squared_error(y_test, y_pred))

Coefficients:  [-1.09769331e-01  3.10203944e-02  2.34159629e-02  2.53148877e+00
 -1.30218325e+01  4.43089807e+00 -9.48304351e-03 -1.38034810e+00
  2.50932526e-01 -1.09450959e-02 -8.70828561e-01  1.26108482e-02
 -5.20051246e-01]
number of nonzeros in θ:  13
Mean-Squared Error: 23.61884
